# Description

This is a comprehensive project focused on the Single Allocation p-Hub Location Problem (SApHLP) with flow-dependent cost. The p-Hub Location Problem is a classic optimization issue in operations research, specifically in transportation and logistics, where the goal is to determine optimal locations for hub facilities to minimize costs.

The SApHLP studied here aims to optimize the allocation of existing demand to the chosen hubs, while considering the inherent economies of scale present in inter-hub connections. In this particular context, the total number of hub facilities to be opened is pre-defined, resulting in a fixed number of possible solutions to the problem.

The study utilizes benchmark problems derived from CAB (25 nodes), TR (55&81 nodes), and Randomly Generated Problems (RGP-100 nodes) datasets. Algorithms were run 10 times for each test problem, with varying initial solutions, ensuring a robust and thorough testing process.

This project incorporates the Simulated Annealing, a meta-heuristic method, inspired by the cooling process of metals, to solve the Single Allocation p-Hub Location Problem (SApHLP). The algorithm seeks an optimal or near-optimal solution for the strategic placement of hub facilities, while also considering the economies of scale in inter-hub connections.

## Simulated Annealing

In [ ]:
#Importing libraries

import os
import numpy as np
import pandas as pd
import copy
import time
import matplotlib.pyplot as plt
import csv
import random 

import math

In [35]:
def initial_sol(n, p):
    '''
    :param n: number of nodes
    :param p: number of hubs
    :return: an array of size n where all nodes are assigned to hubs and the hubs are assigned 
             at indexes of their values
    '''
    hubs = random.sample(range(1, n + 1), p)
    # Assigning a value of None to a variable is one way to reset it to its original, empty state.'
    array = [None] * n 
    #print(hubs)
    array_new = random.choices(hubs, k=n)
    # hubs assign
    for i in hubs:
        array_new[i - 1] = i
    #print(array_new)
    return array_new  

In [36]:
def generate_neighborhood(initial_solution, hubs):
    '''
    Method to generate the neighbourhood of initial_solution containing random assignment of hubs
    :param initial_solution: array of first assignment of hubs that act as the current solution
    :param hubs: array consisting of hubs in the system
    :return: an array of length same as initial_solution with generated randomly and containing the hubs
    '''
    neighborhood = []
    for i in range(len(initial_solution)):
        node = initial_solution[i]
        if node in hubs and node != i+1:
            for hub in hubs:
                if node != hub:
                    new_solution = initial_solution.copy()
                    new_solution[i] = hub
                    neighborhood.append(new_solution)
        elif node not in hubs:
            for hub in hubs:
                new_solution = initial_solution.copy()
                new_solution[i] = hub
                neighborhood.append(new_solution)
    return random.choice(neighborhood)

In [37]:
def flowloc_cost(flow):
    '''
    Method to calculate the unit flow cost between the node to hub flow by appying discount factor 
    :param flow: value of the cost between the node and hub in the current solution
    :return: value of unit flow cost calculated based on respective boundaries 
    '''
    unit_flow_cost = 0
    if flow < 0:
        raise value(error("Flow cannot be non negative"))
    elif flow < 50000:
        unit_flow_cost = 0 + 1 * flow
    elif flow < 100000:
        unit_flow_cost = 10000 + 0.8 * flow
    elif flow < 200000:
        unit_flow_cost = 30000 + 0.6 * flow
    elif flow >= 200000:
        unit_flow_cost = 70000 + 0.4 * flow

    return unit_flow_cost

In [38]:
def cost_eval_economies_of_scale(array_new, flow_data, cost_data):
    '''
    Method to calculate the total flow cost of the current solution, i.e., array_new by applying discount factor
    :param array_new: an array consisting of n nodes connected to the p hubs
    :param flow_data: a csv file consisting of the flow matrix between all the nodes
    :param cost_data:a csv file consisting of the cost matrix between all the nodes
    :return: the total cost of the solution present in given array_new 
    '''
    total_cost = 0
    hubs = set(array_new)
    for hub_i in hubs:
        for hub_j in hubs:
            inter_hub_flow = 0
            if hub_i != hub_j:
                for i in range(1, len(array_new) + 1):
                    for j in range(1, len(array_new) + 1):
                        if array_new[i - 1] == hub_i and array_new[j - 1] == hub_j:
                            inter_hub_flow += flow_data[i][j]

                total_cost += flowloc_cost(inter_hub_flow) * (cost_data[hub_i][hub_j])
    #                 print (total_cost)

    return total_cost

In [39]:
def node_to_hub_cost(array_new, flow_data, cost_data):
    total_cost = 0
    hubs = set(array_new)
    for hub_i in hubs:
        for hub_j in hubs:
            for i in range(1,len(array_new)+1):
                for j in range(1, len(array_new)+1):
                    if array_new[i-1]==hub_i and array_new[j-1] == hub_j:
                        total_cost += flow_data[i][j]*(cost_data[i][hub_i]+cost_data[hub_j][j])
                        
    return total_cost


In [40]:
def evaluate_neighborhood(neighbour, flow_data, cost_data):
    '''
    Method to calculate the node to hub and scaled economy cost for the given neighbour array
    :param neighbour: consists of an array of nodes that are randomly assigned to the hubs
    :param flow_data: a csv file consisting of the flow matrix between all the nodes
    :param cost_data: a csv file consisting of the cost matrix between all the nodes
    :return: the total cost of the economy of the given neighbourhood (node-hub + scaled costs)
    '''
    scaled_economy = cost_eval_economies_of_scale(neighbour, flow_data, cost_data)
    cost_node_to_hub = node_to_hub_cost(neighbour, flow_data, cost_data)
    cost_eval = cost_node_to_hub + scaled_economy
    return cost_eval

In [41]:
def AP(current_cost, candidate_cost, T):
    '''
    Method to calculate the acceptance probability between the current and candidate costs at a given temperature T
    :param current_cost: The cost of the solution that is being considered initially 
    :param candidate_cost: Cost of the candidate solution that is generated at random 
    :param T: Temperature at which the probability has to be calculated
    :return: Based on which cost is higher, the method returns either the calculated probability or 1 
    '''
    if candidate_cost < current_cost:
        return 1.0
    else:
        return math.exp((current_cost - candidate_cost) / T)

In [19]:
#CAB25 - 3 hubs

#load flow data
flow_data_CAB25 = pd.read_csv('CAB25.csv', skiprows = 0, nrows=25, header=None)
flow_data_CAB25.index +=1
flow_data_CAB25.columns += 1


#Load unit cost data
cost_data_CAB25 = pd.read_csv('CAB25.csv', skiprows = 26, nrows=25, header=None)
cost_data_CAB25.index += 1
cost_data_CAB25.columns +=1


def anneal(n, p, flow_data, cost_data, temperature, min_temp):
    '''
    Method to calculate the most optimal cost using the simulated annealing algorithms
    :param n: the number of the nodes present in the system
    :param p: the number of hubs present in system
    :param flow_data: a csv file consisting of the flow matrix between all the nodes
    :param cost_data: a csv file consisting of the cost matrix between all the nodes
    :param temperature: The highest temperature at which the acceptance probability is calculated
    :param min_temp: The least temperature that is considered for calculating the acceptance probability
    :return: The better solution between the current & candidate solutions, it's cost
              and the total time taken for each iteration
    '''

    start = time.time()
    current_solution = initial_sol(n,p)
    current_cost = evaluate_neighborhood(current_solution, flow_data, cost_data)
    hubs = list(set(current_solution))
    cooling_coeff = 0.9

    while temperature > min_temp:
        for i in range(10):
            candidate_solution = generate_neighborhood(current_solution, hubs) # generate neighbor solution
            candidate_cost = evaluate_neighborhood(candidate_solution, flow_data, cost_data)
            acceptance_probability = AP(current_cost, candidate_cost, temperature)
            random_prob = random.random()
            if acceptance_probability > random_prob:
                current_solution = candidate_solution
                current_cost = candidate_cost
        temperature = temperature * cooling_coeff
    end = time.time()

    return current_solution, current_cost, end-start


# total_hubs = [3]

print("SA CAB 25 Nodes and 3 Hubs")
for p_hub in total_hubs:
    total_iterations = 2
    result = []
    all_costs = []
    #print("p-hub:", p_hub, "\n")    
    for i in range(total_iterations):
        i = i - 1
        #calling the anneal function to execute the SA algorithm
        sim_anneal_tuple = anneal(25, 3, flow_data_CAB25, cost_data_CAB25, 3.0, 0.002)
        all_costs.append(sim_anneal_tuple[0])
        result.append(sim_anneal_tuple)


headings = ['Solution Representation', 'Cost', 'Time taken (s)']
print('\t'.join(headings))
for row in result:
    for x in row:
        print(x, end = ' ')
    print()


SA CAB 25 Nodes and 3 Hubs
Solution Representation	Cost	Time taken (s)
[2, 2, 2, 2, 2, 2, 24, 23, 2, 24, 2, 23, 2, 24, 2, 24, 2, 2, 23, 2, 2, 23, 23, 24, 2]	11192963907.906551	131.84376311302185
[5, 2, 2, 5, 5, 9, 5, 5, 9, 5, 5, 5, 5, 2, 9, 5, 2, 2, 5, 2, 5, 5, 5, 5, 2]	10995548444.583445	129.97934412956238
[13, 18, 18, 13, 13, 18, 13, 19, 18, 13, 13, 19, 13, 13, 13, 13, 18, 18, 19, 18, 13, 19, 19, 13, 18]	9093688963.137606	148.52227520942688
[21, 18, 18, 21, 21, 18, 10, 21, 18, 10, 21, 10, 21, 18, 21, 10, 18, 18, 10, 18, 21, 10, 21, 18, 18]	10276599863.460476	146.34860372543335
[13, 25, 25, 13, 25, 25, 13, 13, 25, 13, 13, 12, 13, 25, 13, 13, 25, 25, 12, 25, 13, 12, 12, 13, 25]	8896256160.57589	142.69727396965027
[1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 23, 5, 1, 5, 1, 5, 5, 23, 5, 5, 23, 23, 1, 5]	11186108423.388489	133.43562483787537
[18, 18, 18, 11, 18, 18, 11, 8, 18, 11, 11, 8, 11, 18, 11, 11, 18, 18, 8, 18, 11, 8, 8, 18, 18]	9446933382.75064	122.17324376106262
[13, 3, 3, 13, 13, 3, 13, 1

In [20]:
# #CAB25 - 5 hubs

#load flow data
flow_data_CAB25 = pd.read_csv('CAB25.csv', skiprows = 0, nrows=25, header=None)
flow_data_CAB25.index +=1
flow_data_CAB25.columns += 1


#Load unit cost data
cost_data_CAB25 = pd.read_csv('CAB25.csv', skiprows = 26, nrows=25, header=None)
cost_data_CAB25.index += 1
cost_data_CAB25.columns +=1


def anneal(n, p, flow_data, cost_data, temperature, min_temp):
    '''
    Method to calculate the most optimal cost using the simulated annealing algorithms
    :param n: the number of the nodes present in the system
    :param p: the number of hubs present in system
    :param flow_data: a csv file consisting of the flow matrix between all the nodes
    :param cost_data: a csv file consisting of the cost matrix between all the nodes
    :param temperature: The highest temperature at which the acceptance probability is calculated
    :param min_temp: The least temperature that is considered for calculating the acceptance probability
    :return: The better solution between the current & candidate solutions, it's cost
              and the total time taken for each iteration
    '''

    start = time.time()
    current_solution = initial_sol(n,p)
    current_cost = evaluate_neighborhood(current_solution, flow_data, cost_data)
    hubs = list(set(current_solution))
    cooling_coeff = 0.9

    while temperature > min_temp:
        for i in range(100):
            candidate_solution = generate_neighborhood(current_solution, hubs) # generate neighbor solution
            candidate_cost = evaluate_neighborhood(candidate_solution, flow_data, cost_data)
            acceptance_probability = AP(current_cost, candidate_cost, temperature)
            random_prob = random.random()
            if acceptance_probability > random_prob:
                current_solution = candidate_solution
                current_cost = candidate_cost
        temperature = temperature * cooling_coeff
    end = time.time()

    return current_solution, current_cost, end-start


total_hubs = [5]

print("SA CAB 25 Nodes and 5 Hubs")
for p_hub in total_hubs:
    total_iterations = 10
    result = []
    all_costs = []
    print("p-hub:", p_hub, "\n")    
    for i in range(total_iterations):
        i = i - 1
        #calling the anneal function to execute the SA algorithm
        sim_anneal_tuple = anneal(25, 5, flow_data_CAB25, cost_data_CAB25, 3.0, 0.002)
        all_costs.append(sim_anneal_tuple[0])
        result.append(sim_anneal_tuple)

headings = ['Solution Representation', 'Cost', 'Time taken (s)']
print('\t'.join(headings))
for row in result:
    for x in row:
        print(x, end = ' ')
    print()




SA CAB 25 Nodes and 5 Hubs
Solution Representation	Cost	Time taken (s)
[21, 21, 21, 21, 21, 21, 7, 21, 21, 7, 21, 12, 21, 21, 21, 7, 21, 21, 12, 21, 21, 22, 23, 21, 21]	11283399353.28436	136.22323417663574
[24, 18, 18, 18, 18, 18, 7, 7, 18, 10, 7, 7, 7, 24, 7, 7, 18, 18, 7, 18, 7, 7, 23, 24, 18]	10406884355.71565	153.4299852848053
[16, 9, 9, 9, 9, 9, 7, 7, 9, 7, 7, 19, 16, 16, 15, 16, 9, 9, 19, 9, 9, 19, 19, 16, 9]	9668553085.462397	140.99165725708008
[13, 17, 17, 21, 21, 17, 13, 21, 21, 13, 21, 21, 13, 14, 21, 16, 17, 17, 21, 17, 21, 21, 21, 14, 17]	9784179174.014442	141.24706196784973
[1, 9, 9, 9, 9, 9, 21, 8, 9, 21, 21, 8, 21, 24, 21, 21, 9, 9, 8, 9, 21, 8, 8, 24, 9]	9410424768.383629	141.9564037322998
[6, 17, 17, 6, 6, 6, 10, 19, 6, 10, 6, 19, 6, 6, 15, 10, 17, 17, 19, 6, 6, 19, 19, 6, 17]	8831836790.484322	140.8938443660736
[13, 2, 17, 4, 4, 4, 13, 8, 4, 13, 4, 8, 13, 13, 4, 13, 17, 17, 8, 2, 4, 8, 8, 13, 2]	8452938476.219443	142.4128954410553
[20, 20, 20, 20, 20, 20, 7, 8, 20, 10

In [21]:
# #TR 55 - 3 hubs

load flow data
flow_data_TR55 = pd.read_csv('TR55.csv', skiprows = 0, nrows=55, header=None)
flow_data_TR55.index +=1
flow_data_TR55.columns += 1

#Load unit cost data
cost_data_TR55 = pd.read_csv('TR55.csv', skiprows = 56, nrows=55, header=None)
cost_data_TR55.index += 1
cost_data_TR55.columns +=1

def anneal(n, p, flow_data, cost_data, temperature, min_temp):
    '''
    Method to calculate the most optimal cost using the simulated annealing algorithms
    :param n: the number of the nodes present in the system
    :param p: the number of hubs present in system
    :param flow_data: a csv file consisting of the flow matrix between all the nodes
    :param cost_data: a csv file consisting of the cost matrix between all the nodes
    :param temperature: The highest temperature at which the acceptance probability is calculated
    :param min_temp: The least temperature that is considered for calculating the acceptance probability
    :return: The better solution between the current & candidate solutions, it's cost
              and the total time taken for each iteration
    '''

    start = time.time()
    current_solution = initial_sol(n,p)
    current_cost = evaluate_neighborhood(current_solution, flow_data, cost_data)
    hubs = list(set(current_solution))
    cooling_coeff = 0.9

    while temperature > min_temp:
        for i in range(100):
            candidate_solution = generate_neighborhood(current_solution, hubs) # generate neighbor solution
            candidate_cost = evaluate_neighborhood(candidate_solution, flow_data, cost_data)
            acceptance_probability = AP(current_cost, candidate_cost, temperature)
            random_prob = random.random()
            if acceptance_probability > random_prob:
                current_solution = candidate_solution
                current_cost = candidate_cost
        temperature = temperature * cooling_coeff
    end = time.time()

    return current_solution, current_cost, end-start


total_hubs = [3]

print("SA TR 55 Nodes and 5 Hubs")
for p_hub in total_hubs:
    total_iterations = 10
    result = []
    all_costs = []
    print("p-hub:", p_hub, "\n")    
    for i in range(total_iterations):
        i = i - 1
        #calling the anneal function to execute the SA algorithm
        sim_anneal_tuple = anneal(55, 3, flow_data_TR55, cost_data_TR55, 4.0, 0.002)
        all_costs.append(sim_anneal_tuple[0])
        result.append(sim_anneal_tuple)

headings = ['Solution Representation', 'Cost', 'Time taken (s)']
print('\t'.join(headings))
for row in result:
    for x in row:
        print(x, end = ' ')
    print()



SA TR 55 Nodes and 5 Hubs
Solution Representation	Cost	Time taken (s)
[31, 54, 54, 31, 31, 54, 7, 7, 54, 54, 31, 7, 7, 31, 54, 7, 54, 7, 54, 31, 31, 54, 54, 31, 31, 7, 7, 31, 31, 7, 31, 7, 7, 54, 54, 54, 31, 54, 31, 7, 54, 54, 31, 7, 31, 31, 31, 54, 31, 54, 54, 7, 31, 54, 54]	33895364658.0	571.7502570152283
[20, 20, 46, 39, 39, 20, 39, 39, 46, 46, 39, 39, 39, 39, 46, 39, 46, 39, 20, 20, 46, 46, 20, 39, 20, 39, 39, 20, 39, 39, 39, 39, 39, 20, 46, 46, 39, 46, 39, 39, 46, 46, 39, 39, 39, 46, 39, 46, 39, 46, 46, 39, 39, 20, 20]	57294471943.8	546.4397518634796
[1, 1, 15, 25, 25, 15, 25, 25, 15, 15, 25, 25, 25, 25, 15, 25, 15, 25, 1, 1, 15, 15, 1, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 1, 15, 15, 1, 15, 25, 25, 15, 15, 25, 25, 25, 15, 25, 15, 25, 15, 15, 25, 25, 1, 1]	48281037160.6	545.3029899597168
[25, 55, 55, 30, 25, 55, 30, 30, 55, 55, 30, 30, 30, 30, 55, 30, 55, 30, 55, 55, 55, 55, 55, 30, 25, 30, 30, 25, 25, 30, 25, 30, 30, 55, 55, 55, 30, 55, 30, 30, 55, 55, 25, 30, 25, 55, 25, 55, 3

In [22]:
#TR 55 - 5 hubs

#load flow data
flow_data_TR55 = pd.read_csv('TR55.csv', skiprows = 0, nrows=55, header=None)
flow_data_TR55.index +=1
flow_data_TR55.columns += 1

#Load unit cost data
cost_data_TR55 = pd.read_csv('TR55.csv', skiprows = 56, nrows=55, header=None)
cost_data_TR55.index += 1
cost_data_TR55.columns +=1

def anneal(n, p, flow_data, cost_data, temperature, min_temp):
    '''
    Method to calculate the most optimal cost using the simulated annealing algorithms
    :param n: the number of the nodes present in the system
    :param p: the number of hubs present in system
    :param flow_data: a csv file consisting of the flow matrix between all the nodes
    :param cost_data: a csv file consisting of the cost matrix between all the nodes
    :param temperature: The highest temperature at which the acceptance probability is calculated
    :param min_temp: The least temperature that is considered for calculating the acceptance probability
    :return: The better solution between the current & candidate solutions, it's cost
              and the total time taken for each iteration
    '''

    start = time.time()
    current_solution = initial_sol(n,p)
    current_cost = evaluate_neighborhood(current_solution, flow_data, cost_data)
    hubs = list(set(current_solution))
    cooling_coeff = 0.9

    while temperature > min_temp:
        for i in range(100):
            candidate_solution = generate_neighborhood(current_solution, hubs) # generate neighbor solution
            candidate_cost = evaluate_neighborhood(candidate_solution, flow_data, cost_data)
            acceptance_probability = AP(current_cost, candidate_cost, temperature)
            random_prob = random.random()
            if acceptance_probability > random_prob:
                current_solution = candidate_solution
                current_cost = candidate_cost
        temperature = temperature * cooling_coeff
    end = time.time()

    return current_solution, current_cost, end-start

total_hubs = [5]

print("SA TR 55 Nodes and 5 Hubs")
for p_hub in total_hubs:
    total_iterations = 10
    result = []
    all_costs = []
    print("p-hub:", p_hub, "\n")    
    for i in range(total_iterations):
        i = i - 1
        #calling the anneal function to execute the SA algorithm
        sim_anneal_tuple = anneal(55, 5, flow_data_TR55, cost_data_TR55, 4.0, 0.002)
        all_costs.append(sim_anneal_tuple[0])
        result.append(sim_anneal_tuple)

headings = ['Solution Representation', 'Cost', 'Time taken (s)']
print('\t'.join(headings))
for row in result:
    for x in row:
        print(x, end = ' ')
    print()


SA TR 55 Nodes and 5 Hubs
Solution Representation	Cost	Time taken (s)
[23, 17, 17, 53, 53, 17, 13, 53, 17, 17, 53, 53, 13, 53, 17, 13, 17, 53, 23, 53, 17, 35, 23, 53, 23, 53, 13, 53, 53, 53, 53, 53, 13, 23, 35, 17, 53, 35, 53, 13, 17, 35, 53, 53, 53, 17, 53, 35, 53, 17, 17, 53, 53, 23, 23]	37151962249.4	652.0876033306122
[19, 19, 35, 7, 7, 35, 7, 7, 35, 35, 7, 7, 7, 39, 35, 7, 19, 7, 19, 39, 39, 35, 23, 7, 19, 7, 7, 19, 19, 7, 7, 7, 7, 19, 35, 35, 39, 35, 39, 7, 35, 19, 39, 7, 19, 39, 19, 35, 39, 35, 35, 7, 39, 19, 19]	35794561139.2	608.6350333690643
[19, 19, 19, 43, 33, 37, 33, 33, 19, 19, 33, 33, 33, 43, 19, 33, 19, 33, 19, 37, 37, 19, 19, 33, 19, 33, 33, 43, 43, 33, 43, 33, 33, 19, 19, 19, 37, 19, 37, 33, 19, 19, 43, 43, 43, 37, 43, 19, 43, 37, 19, 33, 43, 54, 19]	34978724186.8	607.0299689769745
[5, 35, 10, 49, 5, 10, 13, 13, 35, 10, 5, 13, 13, 49, 35, 13, 35, 13, 35, 49, 10, 10, 35, 5, 5, 13, 13, 49, 49, 49, 5, 5, 13, 35, 35, 10, 49, 10, 49, 13, 35, 35, 49, 49, 5, 10, 5, 35, 49, 10

In [23]:
#TR 81 - 5 hubs

#load flow data
flow_data_TR81 = pd.read_csv('TR81.csv', skiprows = 0, nrows=81, header=None)
flow_data_TR81.index +=1
flow_data_TR81.columns += 1

#Load unit cost data
cost_data_TR81 = pd.read_csv('TR81.csv', skiprows = 82, nrows=81, header=None)
cost_data_TR81.index += 1
cost_data_TR81.columns +=1

def anneal(n, p, flow_data, cost_data, temperature, min_temp):
    '''
    Method to calculate the most optimal cost using the simulated annealing algorithms
    :param n: the number of the nodes present in the system
    :param p: the number of hubs present in system
    :param flow_data: a csv file consisting of the flow matrix between all the nodes
    :param cost_data: a csv file consisting of the cost matrix between all the nodes
    :param temperature: The highest temperature at which the acceptance probability is calculated
    :param min_temp: The least temperature that is considered for calculating the acceptance probability
    :return: The better solution between the current & candidate solutions, it's cost
              and the total time taken for each iteration
    '''

    start = time.time()
    current_solution = initial_sol(n,p)
    current_cost = evaluate_neighborhood(current_solution, flow_data, cost_data)
    hubs = list(set(current_solution))
    cooling_coeff = 0.9

    while temperature > min_temp:
        for i in range(100):
            candidate_solution = generate_neighborhood(current_solution, hubs) # generate neighbor solution
            candidate_cost = evaluate_neighborhood(candidate_solution, flow_data, cost_data)
            acceptance_probability = AP(current_cost, candidate_cost, temperature)
            random_prob = random.random()
            if acceptance_probability > random_prob:
                current_solution = candidate_solution
                current_cost = candidate_cost
        temperature = temperature * cooling_coeff
    end = time.time()
    print('current_solution=', current_solution)
    print('current_cost=', current_cost)
    print('time=', end-start)

    return current_solution, current_cost, end-start


total_hubs = [5]

print("SA TR 81 Nodes and 5 Hubs")
for p_hub in total_hubs:
    total_iterations = 10
    result = []
    all_costs = []
    print("p-hub:", p_hub, "\n")    
    for i in range(total_iterations):
        i = i - 1
        #calling the anneal function to execute the SA algorithm
        sim_anneal_tuple = anneal(81, 5, flow_data_TR81, cost_data_TR81, 5.0, 0.002)
        all_costs.append(sim_anneal_tuple[0])
        result.append(sim_anneal_tuple)

headings = ['Solution Representation', 'Cost', 'Time taken (s)']
print('\t'.join(headings))
for row in result:
    for x in row:
        print(x, end = ' ')
    print()



SA TR 81 Nodes and 5 Hubs
Solution Representation	Cost	Time taken (s)
[18, 12, 64, 12, 18, 18, 64, 12, 64, 64, 64, 12, 12, 18, 64, 64, 17, 18, 18, 64, 12, 17, 12, 12, 12, 64, 12, 18, 12, 12, 18, 64, 18, 18, 64, 12, 37, 18, 17, 18, 18, 18, 64, 12, 64, 18, 12, 64, 12, 18, 18, 18, 18, 18, 18, 12, 18, 18, 17, 18, 18, 12, 12, 64, 12, 18, 18, 18, 12, 18, 18, 12, 12, 18, 12, 12, 18, 18, 12, 18, 18]	59934158290.4	1287.9332234859469
[71, 71, 26, 19, 19, 71, 26, 19, 26, 26, 26, 19, 19, 71, 26, 26, 17, 71, 19, 26, 19, 17, 19, 19, 19, 26, 71, 19, 19, 19, 71, 26, 71, 26, 26, 19, 71, 71, 17, 71, 26, 71, 26, 19, 26, 71, 19, 26, 19, 71, 71, 19, 19, 26, 19, 19, 19, 19, 17, 19, 19, 19, 71, 26, 19, 19, 74, 71, 19, 71, 71, 19, 19, 74, 19, 19, 26, 74, 71, 71, 26]	59865416245.6	1374.6281855106354
[46, 46, 71, 69, 5, 71, 7, 69, 7, 71, 71, 69, 69, 71, 7, 71, 71, 71, 5, 7, 46, 71, 46, 69, 69, 71, 46, 5, 69, 46, 46, 7, 46, 71, 7, 69, 71, 71, 71, 71, 71, 71, 71, 46, 7, 46, 46, 7, 69, 71, 71, 5, 69, 71, 5, 46, 5,

In [24]:
#TR 81 - 7 hubs

#load flow data
flow_data_TR81 = pd.read_csv('TR81.csv', skiprows = 0, nrows=81, header=None)
flow_data_TR81.index +=1
flow_data_TR81.columns += 1

#Load unit cost data
cost_data_TR81 = pd.read_csv('TR81.csv', skiprows = 82, nrows=81, header=None)
cost_data_TR81.index += 1
cost_data_TR81.columns +=1


def anneal(n, p, flow_data, cost_data, temperature, min_temp):
    '''
    Method to calculate the most optimal cost using the simulated annealing algorithms
    :param n: the number of the nodes present in the system
    :param p: the number of hubs present in system
    :param flow_data: a csv file consisting of the flow matrix between all the nodes
    :param cost_data: a csv file consisting of the cost matrix between all the nodes
    :param temperature: The highest temperature at which the acceptance probability is calculated
    :param min_temp: The least temperature that is considered for calculating the acceptance probability
    :return: The better solution between the current & candidate solutions, it's cost
              and the total time taken for each iteration
     '''

    start = time.time()
    current_solution = initial_sol(n,p)
    current_cost = evaluate_neighborhood(current_solution, flow_data, cost_data)
    hubs = list(set(current_solution))
    cooling_coeff = 0.9

    while temperature > min_temp:
        for i in range(100):
            candidate_solution = generate_neighborhood(current_solution, hubs) # generate neighbor solution
            candidate_cost = evaluate_neighborhood(candidate_solution, flow_data, cost_data)
            acceptance_probability = AP(current_cost, candidate_cost, temperature)
            random_prob = random.random()
            if acceptance_probability > random_prob:
                current_solution = candidate_solution
                current_cost = candidate_cost
        temperature = temperature * cooling_coeff
    end = time.time()

    return current_solution, current_cost, end-start


total_hubs = [7]

print("SA TR 81 Nodes and 7 Hubs")
for p_hub in total_hubs:
    total_iterations = 10
    result = []
    all_costs = []
    print("p-hub:", p_hub, "\n")    
    for i in range(total_iterations):
        i = i - 1
        #calling the anneal function to execute the SA algorithm
        sim_anneal_tuple = anneal(81, 7, flow_data_TR81, cost_data_TR81, 5.0, 0.002)
        all_costs.append(sim_anneal_tuple[0])
        result.append(sim_anneal_tuple)

headings = ['Solution Representation', 'Cost', 'Time taken (s)']
print('\t'.join(headings))
for row in result:
    for x in row:
        print(x, end = ' ')
    print()




SA TR 81 Nodes and 7 Hubs
Solution Representation	Cost	Time taken (s)
[70, 12, 64, 12, 54, 54, 64, 12, 64, 16, 54, 12, 12, 54, 64, 16, 16, 54, 54, 64, 12, 59, 12, 12, 12, 16, 70, 12, 12, 12, 70, 64, 70, 54, 64, 12, 54, 70, 59, 70, 54, 70, 64, 12, 64, 70, 12, 64, 12, 70, 70, 54, 12, 54, 54, 12, 54, 70, 59, 70, 12, 12, 12, 64, 12, 70, 54, 70, 12, 70, 54, 12, 12, 54, 12, 12, 77, 54, 70, 70, 54]	52333176147	1577.273259
[40, 12, 32, 4, 71, 71, 7, 12, 32, 32, 71, 12, 12, 71, 32, 71, 32, 71, 71, 32, 12, 71, 12, 12, 12, 71, 40, 57, 12, 12, 40, 32, 40, 71, 32, 12, 71, 40, 71, 40, 71, 40, 32, 12, 32, 40, 12, 32, 12, 40, 40, 57, 57, 71, 57, 12, 57, 40, 71, 71, 57, 12, 12, 32, 12, 40, 71, 40, 12, 40, 71, 12, 12, 71, 12, 4, 71, 71, 40, 40, 71]	58111919318	1535.540254
[80, 2, 26, 75, 26, 26, 26, 75, 9, 45, 26, 2, 2, 26, 26, 26, 26, 26, 26, 9, 2, 39, 2, 2, 75, 26, 80, 75, 75, 2, 80, 26, 80, 26, 45, 75, 26, 80, 39, 26, 26, 26, 26, 2, 45, 80, 2, 9, 2, 80, 80, 26, 75, 26, 26, 2, 26, 80, 39, 80, 75, 2, 2

In [25]:
# #RGP 100 - 7 hubs
#load flow data
flow_data_RGP100 = pd.read_csv('RGP100.csv', skiprows = 0, nrows=100, header=None)
flow_data_RGP100.index +=1
flow_data_RGP100.columns += 1

#Load unit cost data
cost_data_RGP100 = pd.read_csv('RGP100.csv', skiprows = 101, nrows=100, header=None)
cost_data_RGP100.index += 1
cost_data_RGP100.columns +=1

def anneal(n, p, flow_data, cost_data, temperature, min_temp):
    '''
    Method to calculate the most optimal cost using the simulated annealing algorithms
    :param n: the number of the nodes present in the system
    :param p: the number of hubs present in system
    :param flow_data: a csv file consisting of the flow matrix between all the nodes
    :param cost_data: a csv file consisting of the cost matrix between all the nodes
    :param temperature: The highest temperature at which the acceptance probability is calculated
    :param min_temp: The least temperature that is considered for calculating the acceptance probability
    :return: The better solution between the current & candidate solutions, it's cost
              and the total time taken for each iteration
   '''
    start = time.time()
    current_solution = initial_sol(n,p)
    current_cost = evaluate_neighborhood(current_solution, flow_data, cost_data)
    hubs = list(set(current_solution))
    cooling_coeff = 0.9

    while temperature > min_temp:
        for i in range(100):
            candidate_solution = generate_neighborhood(current_solution, hubs) # generate neighbor solution
            candidate_cost = evaluate_neighborhood(candidate_solution, flow_data, cost_data)
            acceptance_probability = AP(current_cost, candidate_cost, temperature)
            random_prob = random.random()
            if acceptance_probability > random_prob:
                current_solution = candidate_solution
                current_cost = candidate_cost
        temperature = temperature * cooling_coeff
    end = time.time()

    return current_solution, current_cost, end-start

total_hubs = [7]

print("RGP 100 Nodes and 7 Hubs")
for p_hub in total_hubs:
    total_iterations = 10
    result = []
    all_costs = []
    print("p-hub:", p_hub, "\n")    
    for i in range(total_iterations):
        i = i - 1
        #calling the anneal function to execute the SA algorithm
        sim_anneal_tuple = anneal(100, 7, flow_data_RGP100, cost_data_RGP100, 6.0, 0.002)
        all_costs.append(sim_anneal_tuple[0])
        result.append(sim_anneal_tuple)

headings = ['Solution Representation', 'Cost', 'Time taken (s)']
print('\t'.join(headings))
for row in result:
    for x in row:
        print(x, end = ' ')
    print()





RGP 100 Nodes and 7 Hubs
Solution Representation	Cost	Time taken (s)
[3, 24, 3, 24, 7, 3, 7, 3, 3, 95, 11, 49, 7, 11, 3, 95, 11, 11, 3, 95, 21, 95, 24, 24, 7, 24, 24, 21, 24, 95, 95, 95, 7, 95, 24, 11, 21, 49, 95, 7, 7, 49, 3, 24, 3, 21, 95, 21, 49, 3, 11, 24, 11, 21, 7, 3, 24, 3, 3, 7, 3, 7, 95, 7, 95, 7, 24, 95, 95, 49, 95, 3, 7, 21, 11, 7, 21, 21, 49, 3, 49, 95, 49, 95, 95, 11, 7, 95, 95, 11, 7, 24, 21, 95, 95, 21, 24, 7, 7, 95]	139212667439	2311.950086
[58, 48, 58, 58, 48, 58, 48, 58, 49, 26, 58, 49, 26, 51, 49, 23, 26, 51, 26, 58, 48, 51, 23, 26, 48, 26, 58, 58, 26, 58, 48, 58, 58, 26, 49, 58, 48, 49, 58, 26, 48, 49, 48, 49, 26, 58, 23, 48, 49, 26, 51, 23, 48, 23, 26, 48, 26, 58, 59, 48, 48, 48, 26, 48, 58, 48, 48, 51, 26, 49, 58, 49, 49, 23, 48, 23, 48, 58, 48, 48, 49, 58, 49, 48, 58, 49, 23, 51, 48, 48, 58, 49, 48, 48, 58, 48, 49, 48, 48, 26]	138689665951	2237.556903
[23, 72, 8, 4, 68, 8, 62, 8, 68, 8, 4, 4, 8, 68, 72, 23, 68, 72, 68, 4, 8, 4, 23, 4, 8, 8, 72, 62, 4, 62, 62, 72,

In [27]:
# #RGP 100 - 10 hubs
#load flow data
flow_data_RGP100 = pd.read_csv('RGP100.csv', skiprows = 0, nrows=100, header=None)
flow_data_RGP100.index +=1
flow_data_RGP100.columns += 1

#Load unit cost data
cost_data_RGP100 = pd.read_csv('RGP100.csv', skiprows = 101, nrows=100, header=None)
cost_data_RGP100.index += 1
cost_data_RGP100.columns +=1


def anneal(n, p, flow_data, cost_data, temperature, min_temp):
    '''
    Method to calculate the most optimal cost using the simulated annealing algorithms
    :param n: the number of the nodes present in the system
    :param p: the number of hubs present in system
    :param flow_data: a csv file consisting of the flow matrix between all the nodes
    :param cost_data: a csv file consisting of the cost matrix between all the nodes
    :param temperature: The highest temperature at which the acceptance probability is calculated
    :param min_temp: The least temperature that is considered for calculating the acceptance probability
    :return: The better solution between the current & candidate solutions, it's cost
              and the total time taken for each iteration
#     '''
    start = time.time()
    current_solution = initial_sol(n,p)
    current_cost = evaluate_neighborhood(current_solution, flow_data, cost_data)
    hubs = list(set(current_solution))
    cooling_coeff = 0.9

    while temperature > min_temp:
        for i in range(100):
            candidate_solution = generate_neighborhood(current_solution, hubs) # generate neighbor solution
            candidate_cost = evaluate_neighborhood(candidate_solution, flow_data, cost_data)
            acceptance_probability = AP(current_cost, candidate_cost, temperature)
            random_prob = random.random()
            if acceptance_probability > random_prob:
                current_solution = candidate_solution
                current_cost = candidate_cost
        temperature = temperature * cooling_coeff
    end = time.time()

    return current_solution, current_cost, end-start



total_hubs = [10]

print("RGP 100 Nodes and 10 Hubs")
for p_hub in total_hubs:
    total_iterations = 10
    result = []
    all_costs = []
    print("p-hub:", p_hub, "\n")    
    for i in range(total_iterations):
        i = i - 1
        #calling the anneal function to execute the SA algorithm
        sim_anneal_tuple = anneal(100, 10, flow_data_RGP100, cost_data_RGP100, 6.0, 0.002)
        all_costs.append(sim_anneal_tuple[0])
        result.append(sim_anneal_tuple)

headings = ['Solution Representation', 'Cost', 'Time taken (s)']
print('\t'.join(headings))
for row in result:
    for x in row:
        print(x, end = ' ')
    print()



RGP 100 Nodes and 10 Hubs
Solution Representation	Cost	Time taken (s)
[1, 69, 19, 69, 19, 8, 73, 8, 9, 1, 1, 81, 1, 1, 19, 81, 9, 1, 19, 9, 9, 69, 1, 19, 1, 19, 19, 1, 69, 1, 1, 19, 19, 19, 1, 1, 8, 38, 69, 1, 1, 69, 9, 19, 81, 1, 1, 1, 73, 50, 8, 69, 1, 8, 19, 19, 57, 1, 69, 1, 69, 19, 73, 9, 19, 1, 19, 19, 69, 9, 69, 1, 73, 81, 8, 1, 19, 19, 19, 1, 81, 73, 1, 9, 57, 1, 73, 19, 1, 19, 1, 1, 69, 19, 69, 69, 69, 19, 57, 19]	136204663481	2995.130521
[86, 11, 36, 55, 68, 20, 55, 36, 20, 36, 11, 11, 36, 11, 55, 36, 68, 11, 55, 20, 21, 68, 68, 11, 36, 86, 11, 20, 36, 55, 55, 32, 55, 68, 36, 36, 68, 68, 55, 55, 86, 86, 20, 36, 86, 21, 86, 21, 86, 68, 68, 86, 20, 55, 55, 11, 36, 36, 86, 20, 55, 20, 68, 20, 55, 36, 11, 68, 55, 36, 85, 36, 20, 96, 11, 36, 20, 21, 20, 86, 11, 20, 36, 55, 85, 86, 36, 68, 55, 11, 20, 86, 36, 36, 85, 96, 55, 36, 86, 36]	137587203305	2955.29819
[76, 40, 88, 76, 15, 50, 76, 37, 38, 50, 37, 23, 91, 15, 15, 76, 76, 15, 40, 38, 37, 40, 23, 38, 15, 40, 40, 88, 37, 76, 88